In [1]:
from tensorflow.keras import preprocessing , utils
import os
import yaml
import pandas as pd

In [2]:
import gpt4all
from gpt4all import GPT4All
from deep_translator import GoogleTranslator
import asyncio
import sys

In [3]:
import string
import re
import numpy as np
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

# Loading LLM Mistral Model (Rating System Model)

In [4]:
model = GPT4All(device = 'gpu' , model_name = "alaa_ai_model_mistral_v1.9.gguf" , model_path ='C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me/' , allow_download = False)

# Preparing The Seq2Seq Model :

In [5]:
questions = list()
answers = list()

Reading the dataset:

In [6]:
data_from_excel = pd.read_csv('English Dataset/ratings.csv', encoding ='unicode_escape')#,skiprows = 2)
data_from_excel.head(30)

,Topic,Definition of the topic,Rate,Rating
0,cup,cup (s),0,Zero
1,cup,bottle,0,Zero
2,cup,We calibrate with it,1,One
3,cup,It is placed with a cup of coffee,1,One
4,cup,We drink in it,2,Two
5,cup,A cup to drink from,2,Two
6,cup,We put water/juice/tea in it and drink it,2,Two
7,cup,Something made of glass that we drink from,2,Two
8,cup,"Something to put water, juice, tea in",2,Two
9,Veil,Colorful,0,Zero


In [7]:
for i in range(len(data_from_excel['Definition of the topic'])):
    questions.append(data_from_excel['Definition of the topic'][i])
for i in range(len(data_from_excel['Rating'])):
    answers.append( '<START> ' + data_from_excel['Rating'][i] + ' <END>' )
print("Now : " , len(questions))

Now :  480


In [8]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 740


Loading Encoder Model :

In [9]:
enc_model = load_model('enc_model.h5')

In [10]:
enc_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 29)]              0         
                                                                 
 embedding (Embedding)       (None, 29, 200)           148000    
                                                                 
 lstm (LSTM)                 [(None, 200),             320800    
                              (None, 200),                       
                              (None, 200)]                       
                                                                 
Total params: 468,800
Trainable params: 468,800
Non-trainable params: 0
_________________________________________________________________


Loading Decoder Model :

In [11]:
dec_model = load_model('dec_model.h5')

In [12]:
dec_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 3, 200)       148000      ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 200)]        0           []                               
                                                                                            

In [13]:
isQuestionCanBeAnswered = True

In [14]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        if(tokenizer.word_index.get(word) is None):
            isQuestionCanBeAnswered = False
            break
        else:
            tokens_list.append( tokenizer.word_index[ word ] )
            isQuestionCanBeAnswered = True
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=29 , padding='post') , isQuestionCanBeAnswered

In [15]:
target_lang = ''
output = ''

In [39]:
async def response4(g4a):
    # system = "As a tester you Rate any statement on a scale of 1 to 10. if you couldn't ask to repeat the statement again."
    system = "Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."
    user = ''
    # a better way to use an executor:
    with g4a.chat_session(system_prompt = "You are a Rate System can rate any definition from 0 to 10. if your rate is between 0 and 3 respond with '0' , if your rate is between 4 to 7 respond with '1' and if your rate is between 8 to 10 respond with '2'"):
#         is_message_ended = True
#         while is_message_ended == True:
            prompt =  input("\nUser : ")
#             if(prompt == 'goodbye' or prompt == 'exit'):
#                 break
            print("\nAlaa's robot:")
            generator = g4a.generate(prompt, max_tokens = 1000 , streaming = True)
            event_loop = asyncio.get_running_loop()
            has_tokens = True

            def consume(generator):
                nonlocal has_tokens
                try:
                    return next(generator)
                except StopIteration:
                    has_tokens = False

            while has_tokens:
                token = await event_loop.run_in_executor(None, consume, generator)
                if token is not None:
                    yield token

In [40]:
def get_response_in_arabic(g4a):
    with g4a.chat_session():
#         is_message_ended = True
#         while is_message_ended == True:
            prompt =  input("\nUser : ")
#             if(prompt == 'وداعا' or prompt == 'مع السلامة'):
#                 break
            input_text = prompt
            input_text = GoogleTranslator(source='auto', target='en').translate(input_text)
            output = g4a.generate(prompt=input_text, max_tokens =  1000)
            output = GoogleTranslator(source='auto', target = target_lang).translate(output)
            print("\nAlaa's robot:" , output)

In [41]:
def seq2seqModelActivated(question):
    states_values = enc_model.predict( question )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''



    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word



        if sampled_word == 'end' or len(decoded_translation.split()) > 3:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]
        
        print( decoded_translation )

In [43]:
input_text = GoogleTranslator(source='auto', target = 'en').translate(input( 'Enter Definition : ' ))
print(input_text)
question  , isQuestionCanBeAnswered= str_to_tokens( input_text )
if isQuestionCanBeAnswered:
    seq2seqModelActivated(question)    
else:
    print("Ai Rating Model is Loading...")
    if(target_lang == 'ar'):
#         print("إذا أردت إنهاء الشات يرجى كتابة وداعاً او مع السلامة")
        get_response_in_arabic(model)
    else:
#         print("If you want to exit or end the chat ... Write 'goodbye' or 'exit'")
        async for token in response4(model):
            print(token, end='', flush=True)
        print()

Enter Definition :  Veil is a scarf that we wear on our heads
Veil is a scarf that we wear on our heads
Ai Rating Model is Loading...

User :  Veil is a scarf that we wear on our heads

Alaa's robot:
 1


In [19]:
print(decoded_translation)

 ai is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind end


In [20]:
splitted_output = decoded_translation.rsplit(' ', 1)[0]

In [21]:
print(splitted_output)

 ai is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind


In [22]:
# Exception Handling
# Append Method will save the last Lines and characters
try:
    getFile = open("write_and_read_predicted_answer.txt" , "w+")
    getFile.write(splitted_output)
except IOError:
        print("Unable to Open or read the required File.")
except :
    print("Some Error has been occured!")
else :
    print("The Data Appended and Written Successfully")
finally:
    getFile.close()
    print("File now is closed!")

The Data Appended and Written Successfully
File now is closed!
